# MedGemma 4B VQA Pipeline Test

**Model**: `google/medgemma-4b-it`  
**Task**: Medical Visual Question Answering on Fetal Ultrasound Images  
**Status**: ❌ FAILED - Gated model requires HuggingFace authentication

## Overview

This notebook documents an attempt to test Google's MedGemma 4B instruction-tuned model for medical visual question answering on fetal ultrasound images. MedGemma is a specialized medical LLM that could potentially provide detailed analysis of ultrasound images.

**Note**: This is a research prototype test only. NOT for clinical use.

## 1. Environment Setup and Imports

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
import pandas as pd
from PIL import Image
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Data Loading

In [ ]:
# Define paths
DATA_DIR = Path(r"C:\Users\elyas\Workspace\PyCharm\fada-v3\data\Fetal Ultrasound Labeled")
IMAGE_DIR = Path(r"C:\Users\elyas\Workspace\PyCharm\fada-v3\data\Fetal Ultrasound\Non_standard_NT")

# Load annotations
excel_path = DATA_DIR / "Non_standard_NT_image_list.xlsx"
df = pd.read_excel(excel_path)

print(f"Total images: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")

## 3. Prepare VQA Data

In [ ]:
# Extract question columns
q_cols = [col for col in df.columns if col.startswith('Q')]
print(f"Questions per image: {len(q_cols)}")

# Build VQA dataset from first 5 images
vqa_data = []
for idx, row in df.head(5).iterrows():
    img_path = IMAGE_DIR / row['Image Name']
    if not img_path.exists():
        continue
    
    for q_col in q_cols:
        question = q_col.split('\n', 1)[1] if '\n' in q_col else q_col
        question = question[:100]
        answer = str(row[q_col])
        
        if pd.notna(answer) and answer.lower() not in ['nan', 'none', '']:
            vqa_data.append({
                'image_path': str(img_path),
                'question': question,
                'answer': answer
            })

print(f"Prepared {len(vqa_data)} VQA examples")

## 4. Test Image Loading

In [ ]:
test_img = Image.open(vqa_data[0]['image_path']).convert('RGB')
print(f"Image loaded: {test_img.size}")
test_img

## 5. Model Loading Attempt

**NOTE: This cell will FAIL with authentication error**

In [ ]:
model_id = "google/medgemma-4b-it"

try:
    print(f"Attempting to load model: {model_id}")
    
    processor = AutoProcessor.from_pretrained(model_id)
    print("Processor loaded successfully")
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    print("Model loaded successfully")
    
except Exception as e:
    print(f"\n❌ ERROR: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    print("\n" + "="*80)
    print("FAILURE ANALYSIS")
    print("="*80)
    print("\nMedGemma is a GATED MODEL that requires:")
    print("  1. HuggingFace account authentication")
    print("  2. Acceptance of model terms of use")
    print("  3. Potential Google approval process")
    print("\nTo access this model:")
    print("  1. Visit: https://huggingface.co/google/medgemma-4b-it")
    print("  2. Accept the license agreement")
    print("  3. Generate HF token: https://huggingface.co/settings/tokens")
    print("  4. Login via CLI: huggingface-cli login")
    print("  5. Wait for approval (may take time)")

## Summary

### Test Result: ❌ FAILED

**Reason**: MedGemma 4B is a gated model requiring HuggingFace authentication and Google approval.

### Error Details
- **Error Type**: `GatedRepoError`
- **HTTP Status**: 401 Client Error
- **Root Cause**: Cannot access gated repository without proper credentials

### Why This Test Was Skipped

1. **Authentication Complexity**: Requires HuggingFace account setup and token generation
2. **Approval Process**: May require Google's approval to access the model
3. **Time Constraints**: Project timeline doesn't allow waiting for gated model approval
4. **Alternative Options**: Other open VLMs available that don't require special access

### Recommendations

1. **For this project**: Continue with open models (BLIP-2, Florence-2, LLaVA)
2. **Future work**: If VQA is needed, try LLaVA-Med or BLIP-2 first
3. **If MedGemma is essential**: Budget time for authentication and approval process